In [3]:
import pandas as pd
import json
import csv
import cplex
import time

In [4]:
def listar(x):
    rawData = []
    start = 0
    for i in range(len(x)):
        if x[i] == ' ' or x[i] == '\n':
            rawData.append(x[start:i])
            start = i + 1
    return rawData

def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end=" ")
        print()
        
def ler_grafo(u):
    arquive = open(u, 'r')
    content = arquive.read()
    rawData = []
    rawData = listar(content)

    vertices, arestas = [int(v) for v in rawData[:2]]
    edgeData = [json.loads(edge) for edge in rawData[2:]]
    print(str(vertices) + ' vertices, ' + str(arestas) + ' arestas, ')
    # print(edgeData)
    m1, m2 = [], []
    for i in range(vertices):
        m1.append([0]*vertices)
        m2.append([])
    
    for i in range(0,len(edgeData)-1, 2):
        m1[edgeData[i]][edgeData[i+1]] = 1
        m1[edgeData[i+1]][edgeData[i]] = 1

    if len(m1) != 0:
        for i in  range(len(m1)):
            for j in range(len(m1[i])):
                if m1[i][j] == 1:
                    m2[i].append(j+1)
    # print_matrix(m1)
    # print()
    # print_matrix(m2)
    return m2,vertices,arestas



In [5]:
def resolve_modelo(m2,vertices,arestas,k):
    
    cpx = cplex.Cplex()

    y = cpx.variables.add(obj=[0] * vertices,
                             lb=[0] * vertices, ub=[1] * vertices,
                             types=['B'] * vertices,
                             names=['y_%d' % (p) for p in range(1,vertices+1)])
    h = cpx.variables.add(obj=[1] * vertices,
                            lb=[0] * vertices, ub=[1] * vertices,
                            types=['B'] * vertices,
                            names=['h_%d' % (z) for z in range(1,vertices+1)])

    #Restrição 2
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([y[i] for i in range(vertices)], [1.0]*vertices)],
        senses=['E'],
        rhs= [k])


    #Restrição 3
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]], [1.0,-1.0])
                  for i in range(vertices)],
        senses=['L'] * vertices,
        rhs=[0.0] * vertices)
    
    
    #Restrição 4
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[j-1]], [1.0,-1.0])
                  for i in range(vertices) for j in m2[i]],
        senses=['L'] * (arestas*2),
        rhs=[0.0] * (arestas*2))
    
    
    #Restrição 5
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]]+[y[j-1] for j in m2[i]], [1.0,-1.0]+[-1.0]*len(m2[i]))
                  for i in range(vertices)],
        senses=['G'] * vertices,
        rhs=[-len(m2[i]) for i in range(vertices)])

    
    cpx.parameters.threads.set(1)
    cpx.parameters.timelimit.set(300)
    cpx.objective.set_sense(cpx.objective.sense.maximize)
    
    cpx.write('model.lp')
    start_time = time.time()
    cpx.solve()
    my_time = time.time() - start_time
    status = cpx.solution.get_status()
    # if status != 101:
    #     relaxation = [cpx.solution.MIP.get_best_objective(),
    #                     cpx.solution.MIP.get_cutoff,
    #                     # cpx.solution.MIP.get_incubent_node(),
    #                     # cpx.solution.MIP.get_mip_relative_gap(),
    #                     cpx.solution.MIP.get_num_cuts(cpx.solution.MIP.cut_type.cover),
    #                     cpx.solution.MIP.get_subproblem_status()]
    # else:
    relaxation = []

    return status, cpx.solution.get_objective_value(), cpx.solution.MIP.get_best_objective(), ','.join(relaxation), cpx.solution.get_objective_value(), my_time


nome, status, relaxação linear nó0, best bound, best solution, tempo decorrido, nós

para um conjunto varia k

instancias randomicas , conexo, parametro densidade

In [6]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}

for log_n in range(2,15):
    n = 2**log_n
    for graphClass in ["star", "cubic", "bipartite", "BA", "lollipop"]:
        for k in range(1,4):
            k = 2**(log_n + k - 4)
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k))
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')

                index += 1
            results.to_csv(f'./results3.csv')
print("Fim.")


Default row names c1, c2 ... being created.


Começando graphs/star/star_4.txt_k:0.5.------------------------>
4 vertices, 3 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               300
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 15 rows and 8 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)


Default row names c1, c2 ... being created.


------------------------>graphs/star/star_4.txt_k:0.5 feito.
Começando graphs/star/star_4.txt_k:1.------------------------>
4 vertices, 3 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               300


Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 4 times.
MIP Presolve eliminated 10 rows and 4 columns.
MIP Presolve modified 9 coefficients.
Aggregator did 4 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.05 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.05 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.05 ticks)


...

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               300
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
MIP Presolve modified 16384 coefficients.
Reduced MIP has 65534 rows, 32768 columns, and 163833 nonzeros.
Reduced MIP has 32768 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.70 sec. (907.62 ticks)
Found incumbent of value 2047.000000 after 1.77 sec. (934.08 ticks)
Probing fixed 1 vars, tightened 0 bounds.
Probing time = 0.08 sec. (11.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 16384 rows and 1 columns.
Reduced MIP has 49150 rows, 32767 columns, and 131065 nonzeros.
Reduced MIP has 32767 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 2.26 sec. (3870.94 ticks)
Probing time = 0.03 sec. (23.78 ticks)
Clique table members: 32766.
MIP emphasi

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               300
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
MIP Presolve modified 16384 coefficients.
Reduced MIP has 65534 rows, 32768 columns, and 163833 nonzeros.
Reduced MIP has 32768 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.53 sec. (907.62 ticks)
Found incumbent of value 4095.000000 after 1.59 sec. (934.13 ticks)
Probing fixed 1 vars, tightened 0 bounds.
Probing time = 0.09 sec. (11.16 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 16384 rows and 1 columns.
Reduced MIP has 49150 rows, 32767 columns, and 131065 nonzeros.
Reduced MIP has 32767 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 2.49 sec. (3870.94 ticks)
Probing time = 0.02 sec. (23.78 ticks)
Clique table members: 32766.
MIP emphasi

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               300
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
MIP Presolve modified 16384 coefficients.
Reduced MIP has 65534 rows, 32768 columns, and 163833 nonzeros.
Reduced MIP has 32768 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.50 sec. (907.62 ticks)
Found incumbent of value 8191.000000 after 1.60 sec. (934.22 ticks)
Probing fixed 1 vars, tightened 0 bounds.
Probing time = 0.11 sec. (10.75 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 16384 rows and 1 columns.
Reduced MIP has 49150 rows, 32767 columns, and 131065 nonzeros.
Reduced MIP has 32767 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 2.29 sec. (3870.94 ticks)
Probing time = 0.02 sec. (23.78 ticks)
Clique table members: 32766.
MIP emphasi

"--k--"

In [ ]:
index = 0
for log_n in range(3,15):
    n = 2**log_n
    for graphClass in ["cubic"]:
        for k in range(1,3):
            k = 2**(log_n - k)
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            if n >= 1024: continue
            print(f'Começando {filename}_k:{k}.------------------------>')
            m1,m2,v,e = ler_grafo(filename)
            status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m1,m2,v,e,int(k))
            results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
            print(f'------------------------>{filename}_k:{k} feito.')
            index += 1

print("Fim.")
results.to_csv('./resultslollipop.csv')

SyntaxError: invalid syntax (3842707919.py, line 1)

In [6]:
results

,id,graph_class,n,k,status,best_solution,best_bound,linear_relaxation,extra_params,result_ticks,result_time
0,graphs/star/star_4.txt_2,star,4,2,101,1.0,1.0,,,1.0,0.043337
1,graphs/star/star_4.txt_1,star,4,1,101,0.0,0.0,,,0.0,0.018985
2,graphs/cubic/cubic_4.txt_2,cubic,4,2,101,0.0,0.0,,,0.0,0.027783
3,graphs/cubic/cubic_4.txt_1,cubic,4,1,101,0.0,0.0,,,0.0,0.018319
4,graphs/bipartite/bipartite_4.txt_2,bipartite,4,2,101,0.0,0.0,,,0.0,0.021374
...,...,...,...,...,...,...,...,...,...,...,...
71,graphs/star/star_512.txt_128,star,512,128,101,127.0,127.0,,,127.0,0.166321
72,graphs/cubic/cubic_512.txt_256,cubic,512,256,101,252.0,252.0,,,252.0,2.341433
73,graphs/cubic/cubic_512.txt_128,cubic,512,128,101,124.0,124.0,,,124.0,0.711084
74,graphs/bipartite/bipartite_512.txt_256,bipartite,512,256,101,0.0,0.0,,,0.0,25.290181
